In [1]:
import os
import sys
sys.path.append(os.path.dirname((os.path.abspath(''))))

from src.pipeline import Pipeline
from src.preprocessing.preprocessing import removal, tokenization, stemming

[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pipeline = Pipeline()

In [3]:
pipeline.setup()

In [4]:
pipeline.preprocess()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:01<00:00, 843.92it/s]


In [5]:
pipeline.create_tfidf_embeddings()

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:00<00:00, 9380.74it/s]


In [6]:
pipeline.create_tfidf_feature()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:01<00:00, 775.80it/s]


In [7]:
pipeline.create_jaccard_feature()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:00<00:00, 1626.54it/s]


In [9]:
pipeline.features

,qID,pID,tfidf_cosine,tfidf_euclidean,tfidf_manhattan,jaccard
0,651052,121251,0.852590,0.542973,1.896362,0.200000
1,485808,136865,0.481687,1.018148,5.600224,0.102564
2,550996,149222,0.455940,1.043130,4.301138,0.130435
3,632071,154108,0.000000,1.000000,6.205483,0.000000
4,726432,159582,0.562177,0.935760,4.339546,0.120000
...,...,...,...,...,...,...
1495,595630,8570301,0.260758,1.215929,4.647269,0.095238
1496,678422,881900,0.634672,0.854784,3.813911,0.041667
1497,1164923,3705384,0.386200,1.107971,5.632179,0.093750
1498,662044,1740760,0.458970,1.040221,4.665828,0.086957
